In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'usecase_4_.xlsx'
data = pd.read_excel(file_path)

# Handle Missing Values
data['Collaborators'] = data['Collaborators'].fillna('None')
data['Results First Posted'] = data['Results First Posted'].fillna('Missing')

# Handle Categorical Columns
# Encode Study Status
data = pd.get_dummies(data, columns=['Study Status', 'Sex', 'Funder Type'], drop_first=True)

# Process | Separated Columns
def count_items(column):
    return column.fillna('').apply(lambda x: len(str(x).split('|')))

data['Conditions Count'] = count_items(data['Conditions'])
data['Interventions Count'] = count_items(data['Interventions'])
data['Locations Count'] = count_items(data['Locations'])
data['Phases Count'] = count_items(data['Phases'])

# One-hot encode keywords in Conditions, Interventions, etc.
data['Has DRUG Intervention'] = data['Interventions'].str.contains('DRUG', na=False).astype(int)

# Extract Features from Study Design
def extract_design_features(design):
    features = {'Allocation': '', 'Intervention Model': '', 'Masking': '', 'Primary Purpose': ''}
    if pd.notnull(design):
        for part in design.split('|'):
            key, val = part.split(': ', 1)
            features[key] = val.strip()
    return pd.Series(features)

design_features = data['Study Design'].apply(extract_design_features)
data = pd.concat([data, design_features], axis=1).drop(columns=['Study Design'])

# Convert Dates
date_columns = ['Start Date', 'Primary Completion Date', 'Completion Date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Create Derived Date Features
data['Study Duration (Days)'] = (data['Completion Date'] - data['Start Date']).dt.days
data['Start Year'] = data['Start Date'].dt.year
data['Completion Year'] = data['Completion Date'].dt.year

# Drop unnecessary columns
columns_to_drop = ['Study Title', 'Study URL', 'Brief Summary', 'Other Outcome Measures', 'Locations']
data = data.drop(columns=columns_to_drop)

# Ensure Target Column is Numeric
data['Study Recruitment Rate'] = pd.to_numeric(data['Study Recruitment Rate'], errors='coerce')

# Handle Remaining Missing Values
data = data.fillna(0)

# Save the cleaned dataset
data.to_csv('cleaned_clinical_trials.csv', index=False)

print("Dataset prepared and saved successfully!")


Dataset prepared and saved successfully!


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from collections import Counter

# Load the dataset
file_path = 'usecase_4_.xlsx'
data = pd.read_excel(file_path)

# Handle Missing Values
data['Collaborators'] = data['Collaborators'].fillna('None')
data['Results First Posted'] = data['Results First Posted'].fillna('Missing')

# Handle Categorical Columns
# Encode Study Status, Sex, and Funder Type
data = pd.get_dummies(data, columns=['Study Status', 'Sex', 'Funder Type'], drop_first=True)

# Process | Separated Columns
def count_items(column):
    return column.fillna('').apply(lambda x: len(str(x).split('|')))

data['Conditions Count'] = count_items(data['Conditions'])
data['Interventions Count'] = count_items(data['Interventions'])
data['Locations Count'] = count_items(data['Locations'])
data['Phases Count'] = count_items(data['Phases'])

# Frequency Encoding for Conditions
condition_list = data['Conditions'].dropna().str.split('|').explode()
condition_freq = Counter(condition_list)

def average_condition_freq(conditions):
    return np.mean([condition_freq.get(cond, 0) for cond in conditions.split('|')])

data['Avg Condition Frequency'] = data['Conditions'].fillna('').apply(average_condition_freq)

# Frequency Encoding for Interventions
intervention_list = data['Interventions'].dropna().str.split('|').explode()
intervention_freq = Counter(intervention_list)

def average_intervention_freq(interventions):
    return np.mean([intervention_freq.get(interv, 0) for interv in interventions.split('|')])

data['Avg Intervention Frequency'] = data['Interventions'].fillna('').apply(average_intervention_freq)

# Count Specific Intervention Types (e.g., DRUG)
data['Drug Intervention Count'] = data['Interventions'].fillna('').apply(
    lambda x: sum(1 for i in x.split('|') if i.startswith('DRUG'))
)

# Extract Features from Study Design
def extract_design_features(design):
    features = {'Allocation': '', 'Intervention Model': '', 'Masking': '', 'Primary Purpose': ''}
    if pd.notnull(design):
        for part in design.split('|'):
            key, val = part.split(': ', 1)
            features[key] = val.strip()
    return pd.Series(features)

design_features = data['Study Design'].apply(extract_design_features)
data = pd.concat([data, design_features], axis=1).drop(columns=['Study Design'])

# Convert Dates
date_columns = ['Start Date', 'Primary Completion Date', 'Completion Date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Create Derived Date Features
data['Study Duration (Days)'] = (data['Completion Date'] - data['Start Date']).dt.days
data['Start Year'] = data['Start Date'].dt.year
data['Completion Year'] = data['Completion Date'].dt.year

# Drop unnecessary columns
columns_to_drop = ['Study Title', 'Study URL', 'Brief Summary', 'Other Outcome Measures', 'Locations']
data = data.drop(columns=columns_to_drop)

# Ensure Target Column is Numeric
data['Study Recruitment Rate'] = pd.to_numeric(data['Study Recruitment Rate'], errors='coerce')

# Handle Remaining Missing Values
data = data.fillna(0)

# Save the cleaned dataset
data.to_csv('Final_Nest_cleaned_clinical_trials.csv', index=False)

print("Dataset prepared and saved successfully!")


Dataset prepared and saved successfully!


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from collections import Counter

# Load the dataset
file_path = 'usecase_4_.xlsx'
data = pd.read_excel(file_path)

# Handle Missing Values
data['Collaborators'] = data['Collaborators'].fillna('None')
data['Results First Posted'] = data['Results First Posted'].fillna('Missing')

# Handle Categorical Columns
# Encode Study Status, Sex, and Funder Type
data = pd.get_dummies(data, columns=['Study Status', 'Sex', 'Funder Type'], drop_first=True)

# Process | Separated Columns
def count_items(column):
    return column.fillna('').apply(lambda x: len(str(x).split('|')))

data['Conditions Count'] = count_items(data['Conditions'])
data['Interventions Count'] = count_items(data['Interventions'])
data['Locations Count'] = count_items(data['Locations'])
data['Phases Count'] = count_items(data['Phases'])

# Frequency Encoding for Conditions
condition_list = data['Conditions'].dropna().str.split('|').explode()
condition_freq = Counter(condition_list)

def average_condition_freq(conditions):
    if pd.isna(conditions) or conditions == '':
        return 0
    return np.mean([condition_freq.get(cond.strip(), 0) for cond in conditions.split('|')])

data['Avg Condition Frequency'] = data['Conditions'].fillna('').apply(average_condition_freq)

# Frequency Encoding for Interventions
intervention_list = data['Interventions'].dropna().str.split('|').explode()
intervention_freq = Counter(intervention_list)

def average_intervention_freq(interventions):
    if pd.isna(interventions) or interventions == '':
        return 0
    return np.mean([intervention_freq.get(interv.strip(), 0) for interv in interventions.split('|')])

data['Avg Intervention Frequency'] = data['Interventions'].fillna('').apply(average_intervention_freq)

# Count Specific Intervention Types (e.g., DRUG)
data['Drug Intervention Count'] = data['Interventions'].fillna('').apply(
    lambda x: sum(1 for i in x.split('|') if i.startswith('DRUG'))
)

# Extract Features from Study Design
def extract_design_features(design):
    features = {'Allocation': '', 'Intervention Model': '', 'Masking': '', 'Primary Purpose': ''}
    if pd.notnull(design):
        for part in design.split('|'):
            key, val = part.split(': ', 1)
            features[key] = val.strip()
    return pd.Series(features)

design_features = data['Study Design'].apply(extract_design_features)
data = pd.concat([data, design_features], axis=1).drop(columns=['Study Design'])

# Convert Dates
date_columns = ['Start Date', 'Primary Completion Date', 'Completion Date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Create Derived Date Features
data['Study Duration (Days)'] = (data['Completion Date'] - data['Start Date']).dt.days
data['Start Year'] = data['Start Date'].dt.year
data['Completion Year'] = data['Completion Date'].dt.year

# Drop unnecessary columns
columns_to_drop = ['Study Title', 'Study URL', 'Brief Summary', 'Other Outcome Measures', 'Locations']
data = data.drop(columns=columns_to_drop)

# Ensure Target Column is Numeric
data['Study Recruitment Rate'] = pd.to_numeric(data['Study Recruitment Rate'], errors='coerce')

# Handle Remaining Missing Values
data = data.fillna(0)

# Save the cleaned dataset
data.to_csv('PS4_cleaned_clinical_trials.csv', index=False)

print("Dataset prepared and saved successfully!")


Dataset prepared and saved successfully!
